<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4-weeks---both-approaches" data-toc-modified-id="4-weeks---both-approaches-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>4 weeks - both approaches</a></span></li><li><span><a href="#First-8-weeks---both-approaches" data-toc-modified-id="First-8-weeks---both-approaches-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>First 8 weeks - both approaches</a></span></li><li><span><a href="#From-1.11.2021-to-8th-week-of-2022---both-approaches" data-toc-modified-id="From-1.11.2021-to-8th-week-of-2022---both-approaches-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>From 1.11.2021 to 8th week of 2022 - both approaches</a></span></li></ul></div>

# Adding age to the datset - age at the beginning of the usage of the app

In [1]:
import pandas as pd
import numpy as np
import bz2
pd.set_option('display.max_columns', None)
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import matplotlib as mpl
from scipy import stats
from distfit import distfit
import scikit_posthocs as sp
import datetime as dt

pd.options.display.float_format = '{:.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [2]:
# df = pd.read_csv("/home/evida-monika/mhunters/final_data_3.csv.bz2", compression="bz2", sep=",")

df = pd.read_csv("C:/Users/admin/Desktop/mhunters/final_data_4.csv.bz2", compression="bz2", sep=",")
# this is cleaned dataset
# we need to delete another user - ID = 235, because it has 274 sessions in 4 weeks - impossible.

df = df[df['id_users'] != 235]

In [3]:
# changing types of data

cols = ['name_en_exercises','discarded_session_execution', 'code_name_sessions', 
        'name_en_sessions', 'active_user_programs', 'completed_user_programs',
        'pro_programs', 'available_programs', 'name_en_programs', 
        'gender', 
        'activity_level', 'goal', 'body_type', 'newsletter_subscription', 'notifications_setting',
        'language', 'scientific_data_usage', 'BMI_category', 'name_en_implements']

for col in cols:
    df[col] = df[col].astype('category')
    
col_date = ['updated_at_ex_ex', 'created_at_exercises', 'updated_at_exercises',
            'updated_at_session_execution', 'created_at_user_programs', 'updated_at_user_programs',
            'created_at_programs', 'updated_at_programs', 'created_at_users', 'updated_at_users',
            'date_of_birth']

for col in col_date:
    df[col] = pd.to_datetime(df[col])#.dt.strftime("%Y-%m-%d %H:%M:%S")

cat_names = {
    'gender': {1: 'male', 0: 'female'},
    'activity_level': {0: 'very active', 1: 'active', 2: 'sedentary'},
    'goal': {0: 'lose', 1: 'gain', 2: 'antiaging'},
    'body_type': {0: 'thin', 1: 'mid', 2: 'strong'}
}

df = df.replace(cat_names)


In [4]:
df.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,number_exercises_in_session,number_exercises_without_rest_in_session,exercises_per_user,exercises_without_rest_per_user
0,1296434,0.00,2.00,3,2021-11-01 12:05:39.918185,53220.00,3.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.03,0.04,50.95,2463.91,48.31,55,32,52898,40465
1,1296435,3.00,8.00,1,2021-11-01 12:05:39.928907,53221.00,4.00,5398,2020-10-15 12:37:17.966340,2021-10-13 09:46:29.957461,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (right),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:46.135605,female,1981-12-31,163.00,66.00,active,lose,mid,15.00,False,True,3.00,es,439070.00,True,8.00,0.00,No implement,24.84,Normal,2021/44,51.00,988.00,368.00,46208.00,9680.00,189.80,16.47,22.35,0.13,0.54,50.95,2463.91,48.31,55,32,52898,40465
2,1296436,3.00,9.00,2,2021-11-01 12:05:39.933210,53221.00,4.00,5399,2020-10-15 12:37:17.976799,2021-10-13 09:46:29.967689,"{Piernas,Glúteos}","{glúteos,"" cuádriceps""}","{cadera,"" rodillas"","" tobillos""}",3.50,Lunges (left),8261.00,3.00,6132.00,4.00,3.00,368.00,2021-11-01 12:05:40.078024,False,710.00,2.00,1105.00,PM1.2_V2,Session 2,7181.00,2021-10-25 11:04:40.657070,2021-11-28 11:26:22.131395,False,713.00,True,10.00,2020-11-23 13:41:46.587265,2021-09-29 14:53:34.897432,False,True,1.00,3.00,1.00,2.00,2.00,Get motivated!,The ultimate beginner’s program designed to he...,2273.00,2021-10-25 11:04:40.560589,2022-09-07 12:39:

In [5]:
df = df.sort_values(by = 'updated_at_session_execution')

In [6]:
df_users = df.drop_duplicates(subset = ['id_users'], keep = 'first')

In [7]:
df_users.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,number_exercises_in_session,number_exercises_without_rest_in_session,exercises_per_user,exercises_without_rest_per_user
11973,1292266,8.00,27.00,1,2021-11-01 00:06:11.912989,51747.00,2.00,5236,2020-10-15 12:37:15.906971,2022-01-11 08:02:28.410085,"{""Todo el cuerpo""}","{pectorales,"" tríceps"","" cuádriceps""}","{hombros,"" cadera""}",5.00,Burpee,7934.00,7.00,6017.00,5.00,3.00,222.00,2021-11-01 00:06:11.953512,False,595.00,2.00,1238.00,W01_D02,Fundamentals - Session 2,21743.00,2021-10-28 06:17:47.167701,2021-11-19 22:35:44.327916,False,602.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,6557.00,2021-10-28 06:17:47.108449,2022-07-20 06:23:42.531597,male,1990-01-01,180.00,70.00,active,gain,thin,15.00,True,True,4.00,es,26447.00,True,5.00,0.00,No implement,21.60,Normal,2021/44,39.00,1721.00,224.00,72469.00,7792.00,199.79,28.68,7.81,0.45,2.76,81.70,3801.58,97.48,35,18,51936,29648
18837,1292279,0.00,30.00,4,2021-11-01 00:06:14.029733,51751.00,1.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,7938.00,4.00,6018.00,4.00,4.00,120.00,2021-11-01 00:06:14.236997,False,865.00,3.00,630.00,Descanso_activo_3,Rewarding,26176.00,2021-10-28 15:35:59.277604,2021-10-28 15:35:59.277604,False,594.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,7761.00,2021-10-28 06:35:56.542124,2022-09-07 18:28:49.283551,male,1974-08-29,186.00,79.00,active,gain,mid,10.00,True,True,5.00,es,212090.00,False,37.00,0.00,No implement,22.84,Normal,2021/44,133.00,1647.00,121.00,241247.00,36623.00,275.36,27.45,4.41,0.50,0.69,73.80,16051.71,120.69,32,17,529939,333274
18885,1292302,20.00,38.00,1,2021-11-01 00:14:33.468750,51761.00,2.00,5242,2020-10-15 12:37:15.986380,2021-10-13 09:46:28.871998,"{Piernas,Glúteos}","{cuádriceps,"" isquiotibiales"","" glúteos""}","{rodillas,"" tobillos"","" cadera""}",3.50,Squat,7942.00,1.00,6019.00,5.00,3.00,302.00,2021-11-01 00:14:33.713258,False,537.00,2.00,1223.00,Sesion 2,Session 2,19842.00,2021-10-28 06:15:19.240653,2022-01-25 00:40:01.359275,True,565.00,False,29.00,2020-11-23 14:12:13.700871,2021-09-29 14:54:02.270887,True,True,3.00,4.00,3.00,4.00,4.00,Unbreakable,Join Unbreakable to optimize your body from th...,6000.00,20

In [8]:
from dateutil import relativedelta

In [9]:
df_users2 = df_users.copy()

In [10]:
df_users2['date_session_execution'] = pd.to_datetime(df_users2['updated_at_session_execution']).dt.date


In [11]:
df_users2['date_of_birth'] = pd.to_datetime(df_users2['date_of_birth']).dt.date

In [12]:
df_users2['age_in_years'] = (df_users2['date_session_execution'] - df_users2['date_of_birth']) / np.timedelta64(1, 'Y')

In [13]:
df_users2['age_in_years'] = np.ceil(df_users2['age_in_years'])

In [14]:
df_users2.head()

,id_ex_ex,reps_executed_ex_ex,execution_time_ex_ex,order_ex_ex,updated_at_ex_ex,id_session_set_ex,order_session_set_ex,id_exercises,created_at_exercises,updated_at_exercises,body_parts_focused_exercises,muscles_exercises,joints_exercises,met_multiplier_exercises,name_en_exercises,id_session_block_ex,order_session_block_ex,id_session_execution,difficulty_feedback_session_execution,enjoyment_feedback_session_execution,reps_executed_session_execution,updated_at_session_execution,discarded_session_execution,id_sessions,order_sessions,time_duration_sessions,code_name_sessions,name_en_sessions,id_user_programs,created_at_user_programs,updated_at_user_programs,active_user_programs,current_session_id_user_programs,completed_user_programs,id_programs,created_at_programs,updated_at_programs,pro_programs,available_programs,strength_programs,endurance_programs,technique_programs,flexibility_programs,intensity_programs,name_en_programs,description_en_programs,id_users,created_at_users,updated_at_users,gender,date_of_birth,height,weight,activity_level,goal,body_type,body_fat,newsletter_subscription,notifications_setting,training_days_setting,language,points,scientific_data_usage,best_weekly_streak_users,id_implements,name_en_implements,BMI,BMI_category,YYYY/WW,total_sessions_users2,total_time_session_execution,total_reps_session_execution,total_time_users2,total_reps_users2,reps_per_session_users2,total_time_session_execution_min,reps_per_min_session_execution,exercise_execution_time_min,calories,calories_session_execution,total_calories_users2,kcal_per_session_users2,number_exercises_in_session,number_exercises_without_rest_in_session,exercises_per_user,exercises_without_rest_per_user,date_session_execution,age_in_years
11973,1292266,8.00,27.00,1,2021-11-01 00:06:11.912989,51747.00,2.00,5236,2020-10-15 12:37:15.906971,2022-01-11 08:02:28.410085,"{""Todo el cuerpo""}","{pectorales,"" tríceps"","" cuádriceps""}","{hombros,"" cadera""}",5.00,Burpee,7934.00,7.00,6017.00,5.00,3.00,222.00,2021-11-01 00:06:11.953512,False,595.00,2.00,1238.00,W01_D02,Fundamentals - Session 2,21743.00,2021-10-28 06:17:47.167701,2021-11-19 22:35:44.327916,False,602.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,6557.00,2021-10-28 06:17:47.108449,2022-07-20 06:23:42.531597,male,1990-01-01,180.00,70.00,active,gain,thin,15.00,True,True,4.00,es,26447.00,True,5.00,0.00,No implement,21.60,Normal,2021/44,39.00,1721.00,224.00,72469.00,7792.00,199.79,28.68,7.81,0.45,2.76,81.70,3801.58,97.48,35,18,51936,29648,2021-11-01,32.00
18837,1292279,0.00,30.00,4,2021-11-01 00:06:14.029733,51751.00,1.00,5968,2020-10-16 09:33:15.172657,2022-01-21 19:15:56.401181,{},{0},{0},1.00,Rest,7938.00,4.00,6018.00,4.00,4.00,120.00,2021-11-01 00:06:14.236997,False,865.00,3.00,630.00,Descanso_activo_3,Rewarding,26176.00,2021-10-28 15:35:59.277604,2021-10-28 15:35:59.277604,False,594.00,False,30.00,2020-11-23 14:12:50.072945,2021-09-29 14:53:59.484766,True,True,4.00,4.00,5.00,2.00,4.00,Ring Master,The simplest equipment is often the most effec...,7761.00,2021-10-28 06:35:56.542124,2022-09-07 18:28:49.283551,male,1974-08-29,186.00,79.00,active,gain,mid,10.00,True,True,5.00,es,212090.00,False,37.00,0.00,No implement,22.84,Normal,2021/44,133.00,1647.00,121.00,241247.00,36623.00,275.36,27.45,4.41,0.50,0.69,73.80,16051.71,120.69,32,17,529939,333274,2021-11-01,48.00
18885,1292302,20.00,38.00,1,2021-11-01 00:14:33.468750,51761.00,2.00,5242,2020-10-15 12:37:15.986380,2021-10-13 09:46:28.871998,"{Piernas,Glúteos}","{cuádriceps,"" isquiotibiales"","" glúteos""}","{rodillas,"" tobillos"","" cadera""}",3.50,Squat,7942.00,1.00,6019.00,5.00,3.00,302.00,2021-11-01 00:14:33.713258,False,537.00,2.00,1223.00,Sesion 2,Session 2,19842.00,2021-10-28 06:15:19.240653,2022-01-25 00:40:01.359275,True,565.00,False,29.00,2020-11-23 14:12:13.700871,2021-09-29 14:54:02.270887,True,True,3.00,4.00,3.00,4.00,4.00,Un

In [15]:
df_users2 = df_users2.loc[:, ['id_users', 'age_in_years']]

In [16]:
df2 = df.merge(df_users2, how = 'left', on = 'id_users')

In [17]:
df2.shape

(1798735, 87)

In [28]:
# df2.to_csv(r'C:/Users/admin/Desktop/mhunters/final_data_4_5.csv'+ '.bz2', header=True, index=False,compression='bz2')


In [18]:
# dataset for only one of id_session_execution

df_session_exe_only_once = df2.drop_duplicates(subset=['id_session_execution'], keep='first')

In [20]:
# here dropping all of the columns that are not relevant for session_execution

df_session_exe = df_session_exe_only_once.loc[:, ['id_session_execution', 
                                                  'difficulty_feedback_session_execution',
       'enjoyment_feedback_session_execution',
       'reps_executed_session_execution', 'updated_at_session_execution',
       'discarded_session_execution', 'id_sessions', 'order_sessions',
       'time_duration_sessions', 'code_name_sessions', 'name_en_sessions',
       'id_user_programs', 'created_at_user_programs',
       'updated_at_user_programs', 'active_user_programs',
       'current_session_id_user_programs', 'completed_user_programs',
       'id_programs', 'created_at_programs', 'updated_at_programs',
       'pro_programs', 'available_programs', 'strength_programs',
       'endurance_programs', 'technique_programs', 'flexibility_programs',
       'intensity_programs', 'name_en_programs', 'description_en_programs',
       'id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
       'total_time_session_execution', 'total_reps_session_execution',
       'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
       'total_time_session_execution_min', 'reps_per_min_session_execution',
       'exercise_execution_time_min', 'calories_session_execution',
       'total_calories_users2', 'kcal_per_session_users2',
                                          'YYYY/WW', 'age_in_years']]

In [21]:
# dataset only for users

df_users_only_once = df2.drop_duplicates(subset=['id_users'], keep='first')

In [22]:
# here dropping all of the columns that are not relevant for users

df_users_only = df_users_only_once.loc[:, ['id_users', 'created_at_users', 'updated_at_users', 'gender',
       'date_of_birth', 'height', 'weight', 'activity_level', 'goal',
       'body_type', 'body_fat', 'newsletter_subscription',
       'notifications_setting', 'training_days_setting', 'language', 'points',
       'scientific_data_usage', 'best_weekly_streak_users', 'BMI', 'BMI_category', 'total_sessions_users2',
                                          'total_time_users2', 'total_reps_users2', 'reps_per_session_users2',
                                          'total_calories_users2', 'kcal_per_session_users2',
                                           'exercises_per_user', 'exercises_without_rest_per_user',
                                          'age_in_years']]

In [23]:
# Creating a variable where there are 1 when it is the first time that user did a session in a day and 0 if the user already did a session that day

df_session_exe_copy = df_session_exe.copy()

df_session_exe_copy = df_session_exe_copy.sort_values(['id_users', 'updated_at_session_execution'])

df_session_exe_copy['days_a_week_session_exe'] = np.where((df_session_exe_copy['YYYY/WW'] == df_session_exe_copy['YYYY/WW'].shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.year == df_session_exe_copy['updated_at_session_execution'].dt.year.shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.month == df_session_exe_copy['updated_at_session_execution'].dt.month.shift()) &
                                                (df_session_exe_copy['updated_at_session_execution'].dt.day == df_session_exe_copy['updated_at_session_execution'].dt.day.shift()) & 
                                                (df_session_exe_copy['id_users'] == df_session_exe_copy['id_users'].shift()),
                                                0,1)


In [24]:
# grouped table with calculation of days trained a week for every user and training days setting by the user

df_session_exe_copy2 = df_session_exe_copy.loc[:, ['id_users', 'YYYY/WW', 'days_a_week_session_exe']]

df_session_exe_days_setting = df_session_exe_copy2.groupby(['id_users', 'YYYY/WW'])['days_a_week_session_exe'].sum().reset_index()


df_users_training_days_setting = df_users_only.loc[:, ['id_users', 'training_days_setting']]

df_session_exe_days_setting = df_session_exe_days_setting.merge(df_users_training_days_setting, how = 'left', 
                                                                left_on = 'id_users', right_on = 'id_users')


In [25]:
# new column that says if user achieved weekly training goal (1) or not (0)

df_session_exe_days_setting['training_days_setting'] = df_session_exe_days_setting['training_days_setting'].astype('int64')

df_session_exe_days_setting['training_days_goal_achieved'] = np.where(df_session_exe_days_setting['days_a_week_session_exe'] >= df_session_exe_days_setting['training_days_setting'], 1, 0)


In [26]:
# Preparation

df_session_exe_days_setting['YYYY/WW'].min() # min week and year is 2021/44

df_session_exe_days_setting['YYYY/WW'].max() # maximum week and year is 2022/21

# here we create new df with dates (for whole year, week by week)

week_year_table = pd.DataFrame({'Date': pd.date_range('24-05-2021', '28-05-2022')})
week_year_table = week_year_table.assign(weeknum1= week_year_table['Date'].dt.strftime('%Y/%V'))


week_year_table.rename(columns={'weeknum1': 'YYYY/WW'},
           inplace = True)

week_year_table = week_year_table.assign(weeknum2= week_year_table['Date'].dt.strftime('%V'))
week_year_table.rename(columns={'weeknum2': 'WW'},
           inplace = True)

week_year_table = week_year_table.assign(weeknum3= week_year_table['Date'].dt.strftime('%Y'))
week_year_table.rename(columns={'weeknum3': 'YYYY'},
           inplace = True)


week_year_table['YYYY/WW'] = np.where((week_year_table['YYYY/WW'] == '2022/52') & 
                           (week_year_table['Date'].dt.year == 2022) & 
                           (week_year_table['Date'].dt.month == 1) &
                           (week_year_table['Date'].dt.strftime("%V") == '52'),
                          '2021/52', week_year_table['YYYY/WW'] )

week_year_table = week_year_table.drop_duplicates(subset=['YYYY/WW'], keep = 'first').reset_index()

week_year_table.drop(['index'], axis = 1, inplace = True)

# we are copying the old dataframe

df_session_exe_days_setting_all_weeks = df_session_exe_days_setting.copy()

# here we are mergining new dataframe with the dataframe will only dates

df_session_exe_days_setting_all_weeks = df_session_exe_days_setting_all_weeks.merge(week_year_table, how = 'left', left_on = 'YYYY/WW', right_on = 'YYYY/WW')

# this is the funtion for giving dates weekly - from the start date till the end date (it will be used for each user - from first training to the last one)

def expand_dates(ser):
    return pd.DataFrame({'Date': pd.date_range(ser['Date'].min(), ser['Date'].max(), freq='W')})


newdf = df_session_exe_days_setting_all_weeks.groupby(['id_users']) # we are grouping the users and creating new df

newdf = newdf.apply(expand_dates).reset_index() # now we are using the functions

# below we merge new df with previous one, to get all of the weeks
newdf2 = newdf.merge(df_session_exe_days_setting_all_weeks, how = 'outer', left_on = ['Date', 'id_users'], right_on = ['Date', 'id_users'])

newdf2.drop(['level_1'], axis = 1, inplace = True)

newdf3 = newdf2.copy()

# we are dropping old week and year values to create new ones (for nan sake)
newdf3.drop(['YYYY/WW', 'WW', 'YYYY'], axis = 1, inplace = True)

# creation of new time variables
newdf3 = newdf3.assign(weeknum1= newdf3['Date'].dt.strftime('%Y/%V'))

newdf3.rename(columns={'weeknum1': 'YYYY/WW'},
           inplace = True)

newdf3 = newdf3.assign(weeknum2= newdf3['Date'].dt.strftime('%V'))
newdf3.rename(columns={'weeknum2': 'WW'},
           inplace = True)

newdf3 = newdf3.assign(weeknum3= newdf3['Date'].dt.strftime('%Y'))
newdf3.rename(columns={'weeknum3': 'YYYY'},
           inplace = True)


newdf3['YYYY/WW'] = np.where((newdf3['YYYY/WW'] == '2022/52') & 
                           (newdf3['Date'].dt.year == 2022) & 
                           (newdf3['Date'].dt.month == 1) &
                           (newdf3['Date'].dt.strftime("%V") == '52'),
                          '2021/52', newdf3['YYYY/WW'] )

# sorting values by id user and date, to get non-null values before the null one
newdf3.sort_values(['id_users','YYYY/WW', 'Date'], inplace = True, na_position='last')

newdf3 = newdf3.reset_index()

newdf3.drop(['index'], axis = 1, inplace = True)

newdf4 = newdf3.copy()

newdf4 = newdf4.drop_duplicates(subset = ['id_users', 'YYYY/WW'], keep = 'first')

newdf5 = newdf4.copy()

newdf5.columns

newdf5[['days_a_week_session_exe', 'training_days_goal_achieved']] = newdf5[['days_a_week_session_exe','training_days_goal_achieved']].fillna(value=0)

newdf5[['training_days_setting']] = newdf5[['training_days_setting']].fillna(method='ffill')

In [27]:
df_session_exe_days_all_weeks = newdf5.copy() 

df_session_exe_days_all_weeks['week_with_training'] = np.where(df_session_exe_days_all_weeks['days_a_week_session_exe'] != 0,
                                                               1, 
                                                               0)

## 4 weeks - both approaches

In [28]:
# first 4 weeks

df_first_4_weeks = df_session_exe_days_all_weeks.groupby('id_users').head(4).reset_index(drop=True)

In [29]:
number_of_worked_out_weeks_4 = df_first_4_weeks.groupby(['id_users'])['id_users'].size().reset_index(name = 'counts')

df_first_4_weeks = df_first_4_weeks.merge(number_of_worked_out_weeks_4, how = 'left', left_on = 'id_users', right_on = 'id_users')

number_of_weeks_4 = df_first_4_weeks.groupby(['id_users'])['week_with_training'].sum().reset_index(name = 'sum_of_weeks')

df_first_4_weeks = df_first_4_weeks.merge(number_of_weeks_4, how = 'left', left_on = 'id_users', right_on = 'id_users')

df_first_4_weeks_last_week = df_first_4_weeks.drop_duplicates(['id_users'], keep = 'last').loc[:, ['id_users','week_with_training']]

df_first_4_weeks_last_week.rename(columns = {'week_with_training': 'last_week_training'}, inplace = True)

df_first_4_weeks = df_first_4_weeks.merge(df_first_4_weeks_last_week, how = 'left', left_on = 'id_users', right_on = 'id_users')


df_first_4_weeks_users = df_first_4_weeks.drop_duplicates(['id_users'], keep = 'first')

df_first_4_weeks_users2 = df_first_4_weeks_users.copy()

df_first_4_weeks_users2['cens'] = np.where((df_first_4_weeks_users2['counts'] == 4) & (df_first_4_weeks_users2['sum_of_weeks'] >= 2) & (df_first_4_weeks_users2['last_week_training'] == 1),
                                         0,1)

df_first_4_weeks_users['cens'] = np.where((df_first_4_weeks_users['counts'] == 4) & (df_first_4_weeks_users['sum_of_weeks'] == 4),
                                         0,1)

df_first_4_weeks_users.rename(columns={'sum_of_weeks': 'surv_time'}, inplace = True)
df_first_4_weeks_users2.rename(columns={'sum_of_weeks': 'surv_time'}, inplace = True)

df_first_4_weeks_users_subset = df_first_4_weeks_users.loc[:, ['id_users', 'cens', 'surv_time']]
df_first_4_weeks_users2_subset = df_first_4_weeks_users2.loc[:, ['id_users', 'cens', 'surv_time']]

df_users_4_weeks = df_users_only.merge(df_first_4_weeks_users_subset, how = 'left', left_on = 'id_users', right_on = 'id_users')
df_users_4_weeks2 = df_users_only.merge(df_first_4_weeks_users2_subset, how = 'left', left_on = 'id_users', right_on = 'id_users')

df_users_4_weeks['cens'] = df_users_4_weeks['cens'].astype('category')
df_users_4_weeks2['cens'] = df_users_4_weeks2['cens'].astype('category')
df_users_4_weeks['training_days_setting'] = df_users_4_weeks['training_days_setting'].astype('category')
df_users_4_weeks2['training_days_setting'] = df_users_4_weeks2['training_days_setting'].astype('category')

In [30]:
df_users_4_weeks_demo = df_users_4_weeks.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                                 'age_in_years',
                                                 'cens', 'surv_time']]




In [31]:
df_users_4_weeks_demo2 = df_users_4_weeks2.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                                 'age_in_years',
                                                 'cens', 'surv_time']]




In [50]:
df_users_4_weeks_demo.to_csv(r'C:/Users/admin/Desktop/mhunters/4_weeks_approach1.csv', header=True, index=False)


In [51]:
df_users_4_weeks_demo2.to_csv(r'C:/Users/admin/Desktop/mhunters/4_weeks_approach2.csv', header=True, index=False)


## First 8 weeks - both approaches

In [32]:
# first 8 weeks

df_first_8_weeks = df_session_exe_days_all_weeks.groupby('id_users').head(8).reset_index(drop=True)

number_of_worked_out_weeks_8 = df_first_8_weeks.groupby(['id_users'])['id_users'].size().reset_index(name = 'counts')

df_first_8_weeks = df_first_8_weeks.merge(number_of_worked_out_weeks_8, how = 'left', left_on = 'id_users', right_on = 'id_users')

number_of_weeks_8 = df_first_8_weeks.groupby(['id_users'])['week_with_training'].sum().reset_index(name = 'sum_of_weeks')

df_first_8_weeks = df_first_8_weeks.merge(number_of_weeks_8, how = 'left', left_on = 'id_users', right_on = 'id_users')

df_first_8_weeks_last_week = df_first_8_weeks.drop_duplicates(['id_users'], keep = 'last').loc[:, ['id_users','week_with_training']]

df_first_8_weeks_last_week.rename(columns = {'week_with_training': 'last_week_training'}, inplace = True)

df_first_8_weeks = df_first_8_weeks.merge(df_first_8_weeks_last_week, how = 'left', left_on = 'id_users', right_on = 'id_users')

df_first_8_weeks_users = df_first_8_weeks.drop_duplicates(['id_users'], keep = 'first')
df_first_8_weeks_users2 = df_first_8_weeks_users.copy()

df_first_8_weeks_users['cens'] = np.where((df_first_8_weeks_users['counts'] == 4) & (df_first_8_weeks_users['sum_of_weeks'] == 4),
                                         0,1)

df_first_8_weeks_users2['cens'] = np.where((df_first_8_weeks_users2['counts'] == 8) & (df_first_8_weeks_users2['sum_of_weeks'] >= 6) & (df_first_8_weeks_users2['last_week_training'] == 1),
                                         0,1)

df_first_8_weeks_users.rename(columns={'sum_of_weeks': 'surv_time'}, inplace = True)
df_first_8_weeks_users2.rename(columns={'sum_of_weeks': 'surv_time'}, inplace = True)

df_first_8_weeks_users_subset = df_first_8_weeks_users.loc[:, ['id_users', 'cens', 'surv_time']]
df_first_8_weeks_users2_subset = df_first_8_weeks_users2.loc[:, ['id_users', 'cens', 'surv_time']]

df_users_8_weeks = df_users_only.merge(df_first_8_weeks_users_subset, how = 'left', left_on = 'id_users', right_on = 'id_users')
df_users_8_weeks2 = df_users_only.merge(df_first_8_weeks_users2_subset, how = 'left', left_on = 'id_users', right_on = 'id_users')

df_users_8_weeks['cens'] = df_users_8_weeks['cens'].astype('category')
df_users_8_weeks['training_days_setting'] = df_users_8_weeks['training_days_setting'].astype('category')

df_users_8_weeks2['cens'] = df_users_8_weeks2['cens'].astype('category')
df_users_8_weeks2['training_days_setting'] = df_users_8_weeks2['training_days_setting'].astype('category')


In [33]:
df_users_8_weeks_demo = df_users_8_weeks.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                                 'age_in_years',
                                                 'cens', 'surv_time']]


In [34]:
df_users_8_weeks_demo2 = df_users_8_weeks2.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                                 'age_in_years',
                                                 'cens', 'surv_time']]


In [52]:
df_users_8_weeks_demo.to_csv(r'C:/Users/admin/Desktop/mhunters/8_weeks_approach1.csv', header=True, index=False)


In [53]:
df_users_8_weeks_demo2.to_csv(r'C:/Users/admin/Desktop/mhunters/8_weeks_approach2.csv', header=True, index=False)


## From 1.11.2021 to 8th week of 2022 - both approaches

In [35]:
df_session_exe_days_all_weeks['WW'] = df_session_exe_days_all_weeks['WW'].astype('int64')
df_session_exe_days_all_weeks['YYYY'] = df_session_exe_days_all_weeks['YYYY'].astype('int64')

In [36]:
df_session_exe_days_all_weeks['cens_session'] = np.where((df_session_exe_days_all_weeks['WW'] > 8) &
                                (df_session_exe_days_all_weeks['YYYY'] == 2022) & 
                                (df_session_exe_days_all_weeks['YYYY/WW'] != '2021/52'),
                            0,
                            1)

In [37]:
df_session_exe_days_all_weeks_2 = df_session_exe_days_all_weeks[((df_session_exe_days_all_weeks['WW'] <= 8) & (df_session_exe_days_all_weeks['YYYY'] == 2022)) | (df_session_exe_days_all_weeks['YYYY/WW'] == '2021/52') | (df_session_exe_days_all_weeks['YYYY'] == 2021)]


In [38]:
num_trained_weeks = df_session_exe_days_all_weeks_2.groupby(['id_users'])['week_with_training'].sum().reset_index()

num_trained_weeks.rename(columns = {'week_with_training': 'no_trained_week'}, inplace = True)

In [39]:
df_session_exe_days_all_weeks_2 = df_session_exe_days_all_weeks_2.merge(num_trained_weeks, how = 'left', left_on = 'id_users', right_on = 'id_users')


In [40]:
row_count_per_user = df_session_exe_days_all_weeks_2.groupby(['id_users'])['week_with_training'].count().reset_index()
row_count_per_user.rename(columns = {'week_with_training': 'row_count'}, inplace = True)


In [41]:
df_session_exe_days_all_weeks_2 = df_session_exe_days_all_weeks_2.merge(row_count_per_user, how = 'left', left_on = 'id_users', right_on = 'id_users')


In [42]:
df_session_exe_days_all_weeks_2['div'] = df_session_exe_days_all_weeks_2['no_trained_week'] / df_session_exe_days_all_weeks_2['row_count']


In [43]:
df_session_exe_days_all_weeks_2['cens_4'] = np.where((df_session_exe_days_all_weeks_2['div'] == 1) & (df_session_exe_days_all_weeks_2['WW'] == 8),
                                                     0, 1)


In [44]:
df_users_08 = df_session_exe_days_all_weeks_2.drop_duplicates(['id_users'], keep = 'last')

In [45]:
df_users_08.drop(['row_count', 'div', 'cens_session'], axis = 1, inplace = True)

In [46]:
df_users_08.rename(columns= {'cens_4': 'cens',
                             'no_trained_week': 'surv_time'},
                   inplace = True)

In [47]:
df_session_exe_days_all_weeks_2['cens_2'] = np.where((df_session_exe_days_all_weeks_2['div'] >= 0.75) & (df_session_exe_days_all_weeks_2['WW'] == 8),
                                                     0, 1)


df_users_08_2 = df_session_exe_days_all_weeks_2.drop_duplicates(['id_users'], keep = 'last')

df_users_08_2.drop(['row_count', 'div', 'cens_session', 'cens_4'], axis = 1, inplace = True)

df_users_08_2.rename(columns= {'cens_2': 'cens',
                             'no_trained_week': 'surv_time'},
                   inplace = True)

In [48]:
df_users_08_subset = df_users_08.loc[:, ['id_users', 'cens', 'surv_time']]

df_till_08 = df_users_only.merge(df_users_08_subset, how = 'inner', left_on = 'id_users', right_on = 'id_users')


In [49]:
df_users_till_08_demo = df_till_08.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                           'age_in_years',
                                                 'cens', 'surv_time']]

df_users_08_subset2 = df_users_08_2.loc[:, ['id_users', 'cens', 'surv_time']]

df_till_08_2 = df_users_only.merge(df_users_08_subset2, how = 'inner', left_on = 'id_users', right_on = 'id_users')


df_users_till_08_demo_2 = df_till_08_2.loc[:, ['gender', 'height', 'weight', 'activity_level', 'goal', 
                                                 'body_type', 'body_fat', 'newsletter_subscription',
                                                 'notifications_setting', 'training_days_setting', 'BMI',
                                           'age_in_years',
                                                 'cens', 'surv_time']]

In [54]:
df_users_till_08_demo.to_csv(r'C:/Users/admin/Desktop/mhunters/till_08_approach1.csv', header=True, index=False)


In [55]:
df_users_till_08_demo_2.to_csv(r'C:/Users/admin/Desktop/mhunters/till_08_approach2.csv', header=True, index=False)
